In [1]:
import binascii
from helpers import get_input_init_str

In [2]:
# filename = "test_params/123-params.txt"
filename = "313423/313423-parameters.txt"

In [3]:
input_init_str = get_input_init_str(filename, ex=3)
exec(input_init_str)

Reading: '313423/313423-parameters.txt'


In [4]:
def cont_frac(num, den, i = None):
    ''' Function that returns the unique finite sequence [a0, a1, ..., ak] of the terms of the
    continuous fraction of the rational number num/den
    
    Parameters
    ----------
    num : integer
        numerator of the rational number given as input
    den : integer
        denominator of the rational numbert given as input
    i : integer
        required to be non negative; if specified, returns the i-th convergent of num/den; 
        for any i<k the returned list will be of the type [a0, a1, ..., ai]
    '''
    a = num//den
    seq = [a]
    
    if num%den == 0 or i == 0:
        return seq
    
    num -= a*den
    
    if i == None:
        seq += cont_frac(den, num)
    else:
        seq += cont_frac(den, num, i - 1)
                         
    return seq

In [5]:
def sequence_to_frac(seq):
    ''' Given the unique sequence [a0, a1, ..., ak] of the terms of the continuous fraction 
    (or i-th convergent) of a rational number, returns the numerator and denominator of the
    fraction that corresponds to that sequence
    
    Parameters
    ----------
    seq : list
        terms of the continuous fraction (or i-th convergent) of the number
        
    Returns
    -------
    integer, integer
        numerator and denominator of the corresponding rational number
    '''
    
    # Base case: if the lenght is 1, then the continuos fraction contains just the numerator 
    # of the number itself
    if len(seq) == 1:
        return (seq[0], 1)
    
    # Iterative step: the rational number is composed by the first term in the sequence plus
    # the reciprocal of the rational number represented by the remaining subsequence
    num_a_succ, den_a_succ = sequence_to_frac(seq[1:])
    n = seq[0] + den_a_succ/num_a_succ
    return n.numerator(), n.denominator()

In [6]:
def floor_sqrt(x) : 
    ''' Function that computes the square root of a number (rounded to its floor)
    
    Parameters
    ----------
    x : int
        radicand
        
    Returns
    -------
    integer
        floor of the square root of x
    '''
    # Base cases 
    if (x == 0 or x == 1) : 
        return x 
   
    lower = 10**(len(str(x))//2 - 1) 
    upper = 10**(len(str(x))//2 + 1)
    
    while (lower <= upper) : 
        mid = (lower + upper) // 2
          
        # If x is a perfect square 
        if (mid*mid == x) : 
            return mid 
              
        # Since we need floor, we update  
        # answer when mid*mid is smaller 
        # than x, and move closer to sqrt(x) 
        if (mid * mid < x) : 
            lower = mid + 1
            ans = mid 
              
        else : 
              
            # If mid*mid is greater than x 
            upper = mid-1
              
    return ans 

In [7]:
def find_factors(n, phi_n):
    ''' Given n=pq with p and q primes and phi(n) (Euler's phi of n) retrieve p and q
    
    Parameters
    ----------
    n : integer
        product of two prime numbers
    phi_n : integer
        Euler's phi of n
    
    Returns
    -------
    p, q
        prime factors of n
    None 
        if no prime factor has been found
    
    '''
    
    # phi(n) = (p-1)(q-1) = pq-p-q+1 = n-p-q+1 => p+q = n-phi(n)+1
    sum_p_q = n - phi_n + 1
    
    if sum_p_q < 0:
        return None
    
    # Binary search of the factors (assuming that q < p < 2q)
    lower = floor_sqrt(n)
    upper = floor_sqrt(2 * n)
    
    while(lower <= upper):
        
        # Take the middle point
        q = (upper + lower) // 2
        p = sum_p_q - q
        
        if p * q == n:
            # Found them!
            return p, q
        elif p * q > n: 
            # To make the product smaller we want to increase the difference between p and q
            if q < p:
                upper = q - 1
            else:
                lower = q + 1
        elif p * q < n:
            # To make the product bigger we want to decrease the difference between p and q
            if q < p:
                lower = q + 1
            else:
                upper = q - 1
    return None

In [8]:
# Find the continued fraction expansion of e/N
seq = cont_frac(Q3a_e, Q3a_N)

for i in range(len(seq) - 1):
    # Take the ith convergent
    ith_conv = seq[:i+1]
    # Recover the numerator and the denominator of the ith convergent
    hi, ki = sequence_to_frac(ith_conv)
    # If the numerator is non-zero (in that case we cannot compute phi)
    if hi != 0:
        # Compute phi(N)
        candidate_phi_n = (ki * Q3a_e - 1) // hi
        # Factorize N knowing phi(N)
        factors = find_factors(Q3a_N, candidate_phi_n)
        # If the candidate phi(N) was the correct one we should get the two factors (p and q) of N
        if factors != None:
            p, q = factors
            break

In [9]:
# Check whether the product of the found factors is exactly N and then print them
p * q == Q3a_N

True

In [10]:
p

891114096514443467110886545251050219770562940108589030809115405663547004546089043636711997938800052677109094941278989628508610825411687035268906235941698934196434303135837684109487071413884397640134429977017633068052891738472989569

In [11]:
q

1566953785786938622314405339346799628114090389025096510146185928170684968434195118919766345956499836580057584877014639846846056411890541650149349905539057560736567155665544560106509152586382680273420379019041310461217051852399747401

In [12]:
def my_gcd(a, b): 
    ''' Find the gcd of a and b (can be used to find the gcd of two polynomials over a ring)
    '''
    return a.monic() if b == 0 else my_gcd(b, a % b)

In [13]:
def find_related_message(delta, e, n, c1, c2):
    ''' Given that c1 and c2 are the ciphertext generated using an RSA encryption scheme with 
    public exponent e and modulo N, and that their respective plain messages are M1=f(M2) and M2,
    find the integer representation of M2
    
    Parameters
    ----------
    delta : integer
        coefficient with degree 0 of the linear monic polynomial f(x) = x + delta
    e : integer
        exponent of the RSA encryption scheme (must be 3 in order to be able to find M2)
    n : integer
        modulo of the RSA encryption scheme
    c1 : integer
        ciphertext obtained with the message M1
    c2 : integer 
        ciphertext obtained with the message M2
    
    Returns
    -------
    integer
        integer representation of M2
    '''

    P1.<x> = Zmod(n)[]
    f1 = x^e - c1
    f2 = (x + delta)^e - c2
    
    # Return the inverse of the term of degree 0
    return - my_gcd(f1, f2).coefficients()[0] 

In [14]:
def get_plain_msg(e, n, c1, c2):
    ''' Print the message M encrypted with 2 different paddings in c1 and c2 using RSA
    
    Parameters
    ----------
    e : integer
        exponent of the RSA encryption scheme (must be 3 in order to be able to find M)
    n : integer
        modulo of the RSA encryption scheme
    c1 : integer
        ciphertext obtained with the message M and padding r1
    c2 : integer 
        ciphertext obtained with the message M and padding r2
    '''
    
    Z_n = Zmod(n)
    P.<x, y> = PolynomialRing(Z_n)
    P2.<y> = PolynomialRing(Z_n)
    
    g1 = (P(x) ^ e - c1).change_ring(P2)
    g2 = ((P(x) + P(y)) ^ e - c2).change_ring(P2)
    
    # Find the resultant of g1 and g2
    res = g1.resultant(g2, x)
    univ_res = res.univariate_polynomial().change_ring(Z_n)
    
    # delta = r2 - r1 is a "small" root of the resultant polynomial
    # (the parameters of small_roots have been found empirically)
    delta = univ_res.small_roots(beta=1.0, epsilon=0.1)[0]
    
    # Find the message exploiting the knowledge of the difference between the paddings delta
    m2 = find_related_message(delta, e, n, c1, c2)
    
    # Finally, remove the padding by multiplying by m
    k = Integer(Integer(Q3b_N).nbits() // Integer(Q3b_e ** 2))
    m = 2 ** k
    hex_msg = (Integer(m2) // m).hex()   
    msg = binascii.unhexlify(hex_msg).decode()
    return msg

In [15]:
get_plain_msg(Q3b_e, Q3b_N, Q3b_c1, Q3b_c2)

"If I can't even protect my captain's dream, then whatever ambition I have is nothing but talk... (Roronoa Zoro, One Piece)"